# Goal: predict downturns in school performance 
Can we use non-academic school data to predict negative changes in school performance over the next year?



In [1]:
import pickle
import pandas as pd
import numpy as np
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data Preparation


In [2]:
OUTCOME1 = "Student_Growth_Rating"
OUTCOME2 = "Student_Attainment_Rating"
FEATURE_COLS = ["Culture_Climate_Rating", "School_Survey_Student_Response_Rate_Pct", "School_Survey_Teacher_Response_Rate_Pct", "Creative_School_Certification",
                "School_Survey_Involved_Families", "School_Survey_Supportive_Environment", "School_Survey_Ambitious_Instruction", "School_Survey_Effective_Leaders", 
                "School_Survey_Collaborative_Teachers", "School_Survey_Safety", "Suspensions_Per_100_Students_Year_1_Pct", "Suspensions_Per_100_Students_Year_2_Pct", 
                "Misconducts_To_Suspensions_Year_1_Pct", "Misconducts_To_Suspensions_Year_2_Pct", "Average_Length_Suspension_Year_1_Pct", "Average_Length_Suspension_Year_2_Pct", 
                "School_Survey_School_Community", "School_Survey_Parent_Teacher_Partnership", "School_Survey_Quality_Of_Facilities", "Student_Attendance_Year_1_Pct", 
                "Student_Attendance_Year_2_Pct", "Teacher_Attendance_Year_1_Pct", "Teacher_Attendance_Year_2_Pct"]

GENERAL = {'VERY STRONG': 5, 'STRONG': 4, 'NEUTRAL': 3, 'WEAK': 2, 'VERY WEAK': 1, 'NOT ENOUGH DATA': 0}
CATEGORIES = {'Student_Growth_Rating': {'FAR ABOVE AVERAGE': 5, 'ABOVE AVERAGE': 4, 'AVERAGE': 3, 'BELOW AVERAGE': 2, 'FAR BELOW AVERAGE': 1, 'NO DATA AVAILABLE': 0},
'Student_Attainment_Rating': {'FAR ABOVE AVERAGE': 5, 'ABOVE AVERAGE': 4, 'AVERAGE': 3, 'BELOW AVERAGE': 2, 'FAR BELOW AVERAGE': 1, 'NO DATA AVAILABLE': 0},
'Culture_Climate_Rating': {'WELL ORGANIZED': 5, 'ORGANIZED': 4, 'MODERATELY ORGANIZED': 3, 'PARTIALLY ORGANIZED': 2, 'NOT YET ORGANIZED': 1, 'NOT ENOUGH DATA': 0},
'Creative_School_Certification': {'EXCELLING': 4, 'STRONG': 3, 'DEVELOPING': 2, 'EMERGING': 1, 'INCOMPLETE DATA': 0},
'School_Survey_Involved_Families': GENERAL, 
'School_Survey_Supportive_Environment': GENERAL,
'School_Survey_Ambitious_Instruction': GENERAL,
'School_Survey_Effective_Leaders': GENERAL,
'School_Survey_Collaborative_Teachers': GENERAL,
'School_Survey_Safety': GENERAL,
'School_Survey_School_Community': GENERAL,
'School_Survey_Parent_Teacher_Partnership': GENERAL,
'School_Survey_Quality_Of_Facilities': GENERAL,
}

In [3]:
sy1516 = pd.read_csv('CPS1516_shortened.csv')
sy1617 = pd.read_csv('CPS1617_shortened.csv')

In [4]:
# convert school id column to string
sy1516['School_ID']=sy1516['School_ID'].astype('str')
sy1617['School_ID']=sy1617['School_ID'].astype('str')

# convert these columns to upper case to match CATEGORIES dictionary
for i in ['School_Survey_School_Community', 'School_Survey_Parent_Teacher_Partnership', 'School_Survey_Quality_Of_Facilities']:
    sy1516[i]=sy1516[i].str.upper()


sy1516.describe()
sy1516.dtypes

,Progress_Report_Year,Excelerate_Award_Gold_Year,Growth_Reading_Grades_Tested_Pct_ES,Growth_Math_Grades_Tested_Pct_ES,Attainment_Reading_Pct_ES,Attainment_Math_Pct_ES,School_Survey_Student_Response_Rate_Pct,School_Survey_Teacher_Response_Rate_Pct,NWEA_Reading_Growth_Grade_3_Pct,NWEA_Reading_Growth_Grade_4_Pct,...,One_Year_Dropout_Rate_Year_2_Pct,Growth_ACT_Grade_11_Pct,Attainment_ACT_Grade_11_Pct,Freshmen_On_Track_School_Pct_Year_2,Freshmen_On_Track_School_Pct_Year_1,College_Enrollment_School_Pct_Year_2,College_Enrollment_School_Pct_Year_1,College_Persistence_School_Pct_Year_2,College_Persistence_School_Pct_Year_1,Mobility_Rate_Pct
count,670.0,138.0,489.000000,489.000000,492.000000,492.000000,651.000000,651.000000,449.000000,452.000000,...,181.000000,127.000000,144.000000,134.000000,88.000000,154.000000,146.000000,130.000000,105.000000,545.000000
mean,2015.0,2015.0,59.619632,56.492843,50.788618,52.997967,83.113364,82.310292,49.129176,46.606195,...,11.958564,34.559055,27.993056,82.911194,82.188636,51.105195,50.965753,60.783077,65.215238,20.135963
std,0.0,0.0,28.202911,26.547368,31.037262,31.482740,22.067952,14.174477,33.578882,33.073321,...,15.098988,20.773783,23.805886,10.476581,12.197045,24.451304,22.461785,18.428286,15.263952,15.476888
min,2015.0,2015.0,1.000000,1.000000,1.000000,1.000000,0.000000,3.100000,1.000000,1.000000,...,0.000000,2.000000,3.000000,41.700000,51.400000,0.000000,0.000000,14.300000,25.000000,0.400000
25%,2015.0,2015.0,38.000000,36.000000,23.000000,22.750000,76.950000,73.300000,18.000000,16.000000,...,2.200000,16.500000,10.000000,76.400000,74.650000,34.375000,32.625000,49.050000,54.300000,9.400000
50%,2015.0,2015.0,64.000000,58.000000,49.000000,53.000000,91.100000,85.100000,47.000000,44.000000,...,4.800000,34.000000,19.500000,83.800000,84.050000,52.150000,51.900000,63.700000,67.600000,16.200000
75%,2015.0,2015.0,83.000000,78.000000,77.250000,82.000000,98.850000,93.900000,82.000000,79.000000,...,11.500000,46.000000,40.250000,90.800000,91.725000,72.725000,68.875000,71.975000,75.600000,27.800000
max,2015.0,2015.0,99.000000,99.000000,99.000000,99.000000,99.900000,99.900000,99.000000,99.000000,...,59.400000,96.000000,99.000000,100.000000,100.000000,92.200000,89.900000,95.700000,95.700000,141.700000


School_ID                                    object
Short_Name                                   object
Primary_Category                             object
Progress_Report_Year                          int64
Excelerate_Award_Gold_Year                  float64
Student_Growth_Rating                        object
Growth_Reading_Grades_Tested_Pct_ES         float64
Growth_Reading_Grades_Tested_Label_ES        object
Growth_Math_Grades_Tested_Pct_ES            float64
Growth_Math_Grades_Tested_Label_ES           object
Student_Attainment_Rating                    object
Attainment_Reading_Pct_ES                   float64
Attainment_Math_Pct_ES                      float64
Culture_Climate_Rating                       object
School_Survey_Student_Response_Rate_Pct     float64
School_Survey_Teacher_Response_Rate_Pct     float64
Creative_School_Certification                object
NWEA_Reading_Growth_Grade_3_Pct             float64
NWEA_Reading_Growth_Grade_4_Pct             float64
NWEA_Reading

In [5]:
def treat_cats(df, CATEGORIES):
    '''
    Turn categorical rating columns into integers, based on mapping in CATEGORIES
    '''
    for i in df.columns:
        if CATEGORIES.get(i):
            df[i] = df[i].map(CATEGORIES[i])
    return df

In [6]:
sy1516 = treat_cats(sy1516, CATEGORIES)
sy1617 = treat_cats(sy1617, CATEGORIES)

In [7]:
# whittle down the dataframes
sy1516=sy1516.rename(columns={OUTCOME1: OUTCOME1 + '_1516', OUTCOME2: OUTCOME2 + '_1516'})
sy1617=sy1617.rename(columns={OUTCOME1: OUTCOME1 + '_1617', OUTCOME2: OUTCOME2 + '_1617'})

sy1516 = sy1516[["School_ID", "Primary_Category", OUTCOME1+'_1516', OUTCOME2+'_1516'] + FEATURE_COLS]
sy1617 = sy1617[['School_ID', OUTCOME1+'_1617', OUTCOME2+'_1617']]

sy1516.shape
sy1617.shape

(670, 27)

(669, 3)

In [8]:
df = pd.merge(sy1516, sy1617, how='inner', left_on='School_ID', right_on="School_ID")

In [9]:
df.shape
df.head()
print("Percent null:")
df.isnull().sum()/df.shape[0]

(668, 29)

,School_ID,Primary_Category,Student_Growth_Rating_1516,Student_Attainment_Rating_1516,Culture_Climate_Rating,School_Survey_Student_Response_Rate_Pct,School_Survey_Teacher_Response_Rate_Pct,Creative_School_Certification,School_Survey_Involved_Families,School_Survey_Supportive_Environment,...,Average_Length_Suspension_Year_2_Pct,School_Survey_School_Community,School_Survey_Parent_Teacher_Partnership,School_Survey_Quality_Of_Facilities,Student_Attendance_Year_1_Pct,Student_Attendance_Year_2_Pct,Teacher_Attendance_Year_1_Pct,Teacher_Attendance_Year_2_Pct,Student_Growth_Rating_1617,Student_Attainment_Rating_1617
0,400009,ES,4.0,3.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,3.0,3.0,2.0,96.4,95.9,NaN,NaN,3,3
1,400010,HS,2.0,1.0,2.0,77.0,63.9,1.0,3.0,3.0,...,NaN,3.0,3.0,2.0,89.4,88.1,NaN,NaN,2,2
2,400011,ES,3.0,4.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,94.6,94.6,NaN,NaN,0,0
3,400013,HS,3.0,2.0,1.0,93.1,99.9,0.0,2.0,3.0,...,NaN,3.0,3.0,2.0,91.7,87.6,NaN,NaN,2,2
4,400017,MS,3.0,3.0,4.0,99.9,97.4,1.0,3.0,3.0,...,NaN,2.0,2.0,4.0,95.2,95.0,NaN,NaN,2,3


Percent null:


School_ID                                   0.000000
Primary_Category                            0.000000
Student_Growth_Rating_1516                  0.005988
Student_Attainment_Rating_1516              0.005988
Culture_Climate_Rating                      0.005988
School_Survey_Student_Response_Rate_Pct     0.026946
School_Survey_Teacher_Response_Rate_Pct     0.026946
Creative_School_Certification               0.005988
School_Survey_Involved_Families             0.005988
School_Survey_Supportive_Environment        0.005988
School_Survey_Ambitious_Instruction         0.005988
School_Survey_Effective_Leaders             0.005988
School_Survey_Collaborative_Teachers        0.005988
School_Survey_Safety                        0.005988
Suspensions_Per_100_Students_Year_1_Pct     0.241018
Suspensions_Per_100_Students_Year_2_Pct     0.287425
Misconducts_To_Suspensions_Year_1_Pct       0.241018
Misconducts_To_Suspensions_Year_2_Pct       0.287425
Average_Length_Suspension_Year_1_Pct        0.

In [10]:
# filter out four schools where all columns are empty for sy1516
no_data = df[OUTCOME1+'_1516'].isnull()

In [11]:
# filter out schools where outcome variables are "NO DATA AVAILABLE"
no_attainment = (df[OUTCOME1+'_1516'] == 0) | (df[OUTCOME1+'_1617'] == 0)
no_growth = (df[OUTCOME2+'_1516'] == 0) | (df[OUTCOME2+'_1617'] == 0)

In [12]:
df = df[(~no_data) & (~no_attainment) & (~no_growth)] 

In [13]:
# create binary columns for outcome variables: downturn in attainment/growth ratings between the two years
for i in [OUTCOME1, OUTCOME2]:
    df[i+'_diff'] = df[i+'_1617']-df[i+'_1516']
    df[i+'_downturn'] = np.where(df[i+'_diff'] < 0, 1,0)

In [14]:
df.shape
df.head()

(584, 33)

,School_ID,Primary_Category,Student_Growth_Rating_1516,Student_Attainment_Rating_1516,Culture_Climate_Rating,School_Survey_Student_Response_Rate_Pct,School_Survey_Teacher_Response_Rate_Pct,Creative_School_Certification,School_Survey_Involved_Families,School_Survey_Supportive_Environment,...,Student_Attendance_Year_1_Pct,Student_Attendance_Year_2_Pct,Teacher_Attendance_Year_1_Pct,Teacher_Attendance_Year_2_Pct,Student_Growth_Rating_1617,Student_Attainment_Rating_1617,Student_Growth_Rating_diff,Student_Growth_Rating_downturn,Student_Attainment_Rating_diff,Student_Attainment_Rating_downturn
0,400009,ES,4.0,3.0,0.0,NaN,NaN,0.0,0.0,0.0,...,96.4,95.9,NaN,NaN,3,3,-1.0,1,0.0,0
1,400010,HS,2.0,1.0,2.0,77.0,63.9,1.0,3.0,3.0,...,89.4,88.1,NaN,NaN,2,2,0.0,0,1.0,0
3,400013,HS,3.0,2.0,1.0,93.1,99.9,0.0,2.0,3.0,...,91.7,87.6,NaN,NaN,2,2,-1.0,1,0.0,0
4,400017,MS,3.0,3.0,4.0,99.9,97.4,1.0,3.0,3.0,...,95.2,95.0,NaN,NaN,2,3,-1.0,1,0.0,0
5,400021,ES,2.0,2.0,2.0,94.0,87.0,0.0,2.0,3.0,...,95.3,95.2,NaN,NaN,4,2,2.0,0,0.0,0


In [15]:
df[OUTCOME1+'_downturn'].value_counts()
df[OUTCOME1+'_diff'].value_counts()

0    457
1    127
Name: Student_Growth_Rating_downturn, dtype: int64

 0.0    214
 1.0    144
-1.0    107
 2.0     75
 3.0     24
-2.0     17
-3.0      2
-4.0      1
Name: Student_Growth_Rating_diff, dtype: int64

In [17]:
df[OUTCOME2+'_downturn'].value_counts()
df[OUTCOME2+'_diff'].value_counts()

0    473
1    111
Name: Student_Attainment_Rating_downturn, dtype: int64

 0.0    398
-1.0    108
 1.0     74
-2.0      3
 2.0      1
Name: Student_Attainment_Rating_diff, dtype: int64

In [18]:
pickle.dump(df, open("cleaned_df.p", "wb"))